In [ ]:
!pip install transformers sentencepiece datasets torch evaluate seqeval

In [ ]:
!gdown 1YLpUgcKAFmmjJzZgXC3lhoqpZttypLLP
!gdown 1IaCcTcQsloQKU-sUa2Dn0cm_wwmUwbfj
!gdown 1MlbiDyhQhRD90WT8jrNok0iF9x3e6-ZD
!gdown 1V-gPciK_Nic6Oct1fO66Cxwg7jGCpv8q
!gdown 1bOgvfvD4H1I3Z4vBeeCcuHldUxB1jKPQ
!gdown 103eSrJAIeATd1yQHLdfVEQz6xtKEoVMB
!gdown 1S0SPFqXu3xB4ivTMV3Gn4b3yPzRJT-dx
!gdown 144GpwcJ9v-xfhMS8LbR4JnuwxwpIOCXI

In [ ]:
# Skip VnCoreNLP setup - Use Kaggle-compatible chunking
print("=== KAGGLE COMPATIBLE VERSION ===")
print("Skipping VnCoreNLP download due to compatibility issues")
print("Using enhanced fallback chunking strategy instead")

# Create directory for consistency (even though we won't use VnCoreNLP)
import os
os.makedirs("vncorenlp", exist_ok=True)

# Set VnCoreNLP model to None - this will trigger fallback chunking
vncore_nlp_model = None
print("✓ VnCoreNLP disabled - will use fallback chunking")
print("✓ All chunking rules (1, 2, 3) will still work effectively")

In [ ]:
import pandas as pd
import numpy as np
import re
import torch
from transformers import (
    AutoTokenizer, 
    AutoModelForTokenClassification, 
    DataCollatorForTokenClassification,
    TrainingArguments,
    Trainer
)
from datasets import Dataset, DatasetDict
import evaluate
import os
from tqdm.auto import tqdm
from typing import List, Dict, Tuple
import json
# Remove vncorenlp import since we're not using it for Kaggle compatibility

In [ ]:
# Check GPU availability
if torch.cuda.is_available():
    print(f"GPU is ready: {torch.cuda.get_device_name(0)}")
else:
    print("GPU is not available.")

# 1. Configuration

In [ ]:
MODEL_NAME = "microsoft/mdeberta-v3-base"  # mDeBERTa model
MAX_LENGTH = 1024
TRAIN_BATCH_SIZE = 4
EVAL_BATCH_SIZE = 4
GRADIENT_ACCUMULATION = 4
LEARNING_RATE = 2e-5
WEIGHT_DECAY = 0.01
GRAD_NORM = 1.0
NUM_EPOCHS = 50
WARMUP_RATIO = 0.1
LABEL_SMOOTHING_FACTOR = 0.1
OUTPUT_DIR = "./mdeberta_ner_model"

In [ ]:
# Column names in Excel file
CONTENT_COL = "content"
TAGGED_CONTENT_COL = "human_tagged_content"

In [ ]:
# Create output directory
os.makedirs(OUTPUT_DIR, exist_ok=True)

In [ ]:
# Initialize chunking system without VnCoreNLP (Kaggle compatible)
print("=== INITIALIZING KAGGLE-COMPATIBLE CHUNKING SYSTEM ===")
vncore_nlp_model = None  # Explicitly set to None for Kaggle

def smart_chunk_text(text: str, max_length: int = 400) -> List[str]:
    """
    Chunking văn bản theo các quy tắc được định nghĩa (Enhanced Kaggle version)
    
    Quy tắc 1: Các mục trong danh sách là các đơn vị độc lập
    Quy tắc 2: Các khối liên hệ và siêu dữ liệu là không thể chia cắt
    Quy tắc 3: Xử lý Hashtag
    
    Args:
        text: Văn bản cần chunking
        max_length: Độ dài tối đa của mỗi chunk
    
    Returns:
        List các chunks
    """
    chunks = []
    
    # Quy tắc 3: Xử lý hashtag - tách ra và lưu trữ riêng
    hashtag_pattern = r'#\w+'
    hashtags = re.findall(hashtag_pattern, text)
    text_without_hashtags = re.sub(hashtag_pattern, '', text).strip()
    
    # Tách văn bản thành các dòng
    lines = text_without_hashtags.split('\n')
    
    current_chunk = ""
    contact_block = ""
    in_contact_block = False
    
    # Các từ khóa để nhận diện khối liên hệ
    contact_keywords = ['hotline', 'địa chỉ', 'website', 'email', 'zalo', 'facebook', 'tel', 'fax', 'tổng đài']
    
    i = 0
    while i < len(lines):
        line = lines[i].strip()
        
        if not line:
            i += 1
            continue
            
        # Quy tắc 2: Phát hiện khối liên hệ
        line_lower = line.lower()
        is_contact_line = any(keyword in line_lower for keyword in contact_keywords)
        
        # Kiểm tra xem có phải là dòng chứa thông tin liên hệ không
        has_address = bool(re.search(r'(số\s+\d+|đường|phường|quận|tỉnh|thành phố)', line_lower))
        has_org_pattern = bool(re.search(r'<ORG>.*?</ORG>', line))
        has_addr_pattern = bool(re.search(r'<ADDR>.*?</ADDR>', line))
        
        if is_contact_line or has_address or has_org_pattern or has_addr_pattern:
            if not in_contact_block:
                # Bắt đầu khối liên hệ mới
                if current_chunk:
                    chunks.append(current_chunk.strip())
                    current_chunk = ""
                in_contact_block = True
                contact_block = line
            else:
                # Tiếp tục khối liên hệ
                contact_block += " " + line
        else:
            # Kết thúc khối liên hệ nếu đang trong khối
            if in_contact_block:
                chunks.append(contact_block.strip())
                contact_block = ""
                in_contact_block = False
            
            # Quy tắc 1: Xử lý các mục trong danh sách
            if re.match(r'^[\-\*]\s+', line) or re.match(r'^\d+[\.\)]\s+', line):
                # Đây là một mục trong danh sách
                if current_chunk:
                    chunks.append(current_chunk.strip())
                    current_chunk = ""
                
                # Xử lý mục danh sách như một chunk riêng
                list_item = line
                
                # Kiểm tra các dòng tiếp theo có phải là phần tiếp theo của mục này không
                j = i + 1
                while j < len(lines) and lines[j].strip():
                    next_line = lines[j].strip()
                    # Nếu dòng tiếp theo không phải là mục danh sách mới
                    if not (re.match(r'^[\-\*]\s+', next_line) or re.match(r'^\d+[\.\)]\s+', next_line)):
                        # Và không phải khối liên hệ
                        next_line_lower = next_line.lower()
                        is_next_contact = any(keyword in next_line_lower for keyword in contact_keywords)
                        
                        if not is_next_contact:
                            list_item += " " + next_line
                            j += 1
                        else:
                            break
                    else:
                        break
                
                chunks.append(list_item.strip())
                i = j - 1
            else:
                # Văn bản thông thường
                if len(current_chunk) + len(line) + 1 <= max_length:
                    if current_chunk:
                        current_chunk += " " + line
                    else:
                        current_chunk = line
                else:
                    # Chunk hiện tại đã đủ dài, tạo chunk mới
                    if current_chunk:
                        chunks.append(current_chunk.strip())
                    current_chunk = line
        
        i += 1
    
    # Xử lý phần còn lại
    if in_contact_block and contact_block:
        chunks.append(contact_block.strip())
    elif current_chunk:
        chunks.append(current_chunk.strip())
    
    # Quy tắc 3: Xử lý hashtag - nối vào chunk cuối cùng nếu có
    if hashtags and chunks:
        hashtag_text = " " + " ".join(hashtags)
        chunks[-1] += hashtag_text
    
    # Enhanced fallback chunking (without VnCoreNLP)
    final_chunks = []
    for chunk in chunks:
        if len(chunk) <= max_length:
            final_chunks.append(chunk)
        else:
            # Smart Vietnamese sentence splitting
            sentences = re.split(r'[.!?]\s+|[.!?]$|(?<=\.)\s+(?=[A-ZÀÁÂÃÈÉÊÌÍÒÓÔÕÙÚĂĐĨŨƠƯ])', chunk)
            
            current_subchunk = ""
            for sent in sentences:
                sent = sent.strip()
                if not sent:
                    continue
                    
                if len(current_subchunk) + len(sent) + 2 <= max_length:
                    if current_subchunk:
                        current_subchunk += ". " + sent
                    else:
                        current_subchunk = sent
                else:
                    if current_subchunk:
                        final_chunks.append(current_subchunk.strip())
                    
                    # If single sentence is still too long, split by clauses
                    if len(sent) > max_length:
                        clauses = re.split(r',\s+|;\s+|\s+và\s+|\s+hoặc\s+|\s+nhưng\s+|\s+mà\s+', sent)
                        current_subchunk = ""
                        for clause in clauses:
                            clause = clause.strip()
                            if len(current_subchunk) + len(clause) + 2 <= max_length:
                                if current_subchunk:
                                    current_subchunk += ", " + clause
                                else:
                                    current_subchunk = clause
                            else:
                                if current_subchunk:
                                    final_chunks.append(current_subchunk.strip())
                                current_subchunk = clause
                    else:
                        current_subchunk = sent
            
            if current_subchunk:
                final_chunks.append(current_subchunk.strip())
    
    return [chunk for chunk in final_chunks if chunk.strip()]

print("✓ Smart chunking initialized successfully")
print("✓ All 3 chunking rules implemented")
print("✓ Enhanced Vietnamese sentence splitting active")
print("✓ Ready for Kaggle deployment")

In [ ]:
def test_chunking():
    """
    Test chunking function với các ví dụ (Kaggle compatible)
    """
    test_text = """
MEGA LIVE 14.11 - Flash Sale Cực Khủng
- Chăm Sóc Da Cao Cấp 3in1
- Dr.Vip Chăm Sóc Da Lão Hoá ECM
- Điều Trị Mụn Chuyên Sâu

<ORG>NHUNG ROSE Store</ORG>: <ADDR>Số 888 Đường Bắc Sơn, Phường 12, Quận Tân Bình</ADDR>
Hotline: 0942.093.999
Website: www.nhungrose.com

Chúng tôi cam kết mang đến dịch vụ chất lượng cao nhất cho khách hàng.

#SeoulCenter #ThamMyVien #ChămsócDa
"""
    
    chunks = smart_chunk_text(test_text)
    print("=== KAGGLE CHUNKING TEST ===")
    print(f"Generated {len(chunks)} chunks:")
    for i, chunk in enumerate(chunks):
        print(f"\nChunk {i+1} (length: {len(chunk)}):")
        print(f"'{chunk}'")
        print("-" * 50)

# Chạy test chunking
test_chunking()

# Smart Chunking Strategy (Kaggle Compatible)

## Kaggle Optimizations:
- ✅ **No external dependencies** - Removed VnCoreNLP requirement
- ✅ **Enhanced fallback chunking** - Smart Vietnamese sentence splitting
- ✅ **Clause-based splitting** - Handles long sentences intelligently
- ✅ **100% compatible** - Works on any Python environment

## Entity Types được nhận dạng:
- **PER**: Tên người
- **ORG**: Tên tổ chức, công ty, doanh nghiệp
- **ADDR**: Địa chỉ, vị trí địa lý

*Note: Đã loại bỏ nhận dạng số điện thoại (PHONE) khỏi hệ thống*

## Quy tắc chunking được implement:

### Quy tắc 1: Các mục trong danh sách là các đơn vị độc lập
- Nhận diện các dòng bắt đầu bằng dấu gạch đầu dòng (-), dấu hoa thị (*), hoặc số thứ tự
- Coi mỗi mục như một "câu" riêng biệt hoặc một đoạn độc lập
- Tránh nối các mục danh sách với nhau để không tạo ra câu vô nghĩa

### Quy tắc 2: Các khối liên hệ và siêu dữ liệu là không thể chia cắt
- Nhận diện khối thông tin liên hệ dựa trên từ khóa: "Hotline", "Địa chỉ", "Website", "Email", "Zalo", "Facebook"
- Phát hiện patterns như địa chỉ, tên tổ chức
- Giữ nguyên khối liên hệ trong một chunk để đảm bảo ngữ cảnh

### Quy tắc 3: Xử lý Hashtag
- Tách các hashtag (#SeoulCenter #ThamMyVien) ra khỏi văn bản chính
- Nối hashtag vào cuối chunk cuối cùng để cung cấp thêm tín hiệu ngữ cảnh

### Enhanced Vietnamese Processing (without VnCoreNLP)
- Sử dụng regex patterns để tách câu tiếng Việt chính xác
- Phân tách theo dấu phẩy và liên từ khi câu quá dài
- Xử lý chữ cái viết hoa tiếng Việt để phát hiện ranh giới câu

In [ ]:
# Tagged label (removed PHONE)
LABELS = ["PER", "ORG", "ADDR"]

# NER labels with BIO format
NER_LABELS = ["O"] + [f"{prefix}-{label}" for prefix in ["B", "I"] for label in LABELS]
ID2LABEL = {i: label for i, label in enumerate(NER_LABELS)}
LABEL2ID = {label: i for i, label in enumerate(NER_LABELS)}

# 2. prepare dataset

In [ ]:
def extract_entities(tagged_text: str, labels: List[str] = LABELS) -> Tuple[str, List[Dict]]:
    """
    Chuyển văn bản có tag thành (text_cleaned, spans) dùng cho NER trong Label Studio

    Args:
        tagged_text (str): Văn bản có chứa tag như <PER>...</PER>
        labels (List[str]): Danh sách nhãn cần trích xuất

    Returns:
        Tuple[str, List[Dict]]: Văn bản đã loại tag, danh sách span gồm (start, end, text, label)
    """
    spans = []
    text = tagged_text

    for label in labels:
        pattern = fr"<{label}>(.*?)</{label}>"
        for match in re.finditer(pattern, text):
            raw_entity = match.group(1)
            # Tính offset không tính tag
            pre_text = text[:match.start()]
            clean_start = len(re.sub(fr"<[^>]+>", "", pre_text))
            clean_end = clean_start + len(raw_entity)
            spans.append({
                "start": clean_start,
                "end": clean_end,
                "text": raw_entity,
                "labels": [label]
            })

        text = re.sub(pattern, r"\1", text)  # Bỏ tag, giữ lại text

    return text, spans

In [ ]:
def create_bio_labels_from_spans(text, spans):
    """
    Creates character-level BIO labels from spans with stricter validation
    
    Args:
        text: Clean text without tags
        spans: List of entity spans with start, end, text, labels
    
    Returns:
        List of character-level BIO labels
    """
    # Initialize all characters with O label
    bio_labels = ["O"] * len(text)
    
    # Sort spans by start position to ensure proper BIO order
    sorted_spans = sorted(spans, key=lambda x: x["start"])
    
    # Add entity labels
    for span in sorted_spans:
        start = span["start"]
        end = span["end"]
        label = span["labels"][0]
        
        # Check if indices are valid
        if start < 0 or end > len(text) or start >= len(text) or end <= 0:
            print(f"Warning: Invalid span indices ({start}, {end}) for text of length {len(text)}")
            continue
        
        # Check for overlapping spans
        if any(l != "O" for l in bio_labels[start:end]):
            print(f"Warning: Overlapping span at position ({start}, {end})")
            # Handle overlapping spans - prioritize this span
            # You could also choose to skip or merge spans based on your requirements
        
        # Set beginning token
        bio_labels[start] = f"B-{label}"
        
        # Set inside tokens
        for i in range(start + 1, end):
            bio_labels[i] = f"I-{label}"
    
    # Validate for any inconsistencies
    for i in range(1, len(bio_labels)):
        if bio_labels[i].startswith("I-"):
            entity_type = bio_labels[i][2:]
            if not (bio_labels[i-1].startswith("B-" + entity_type) or 
                   bio_labels[i-1].startswith("I-" + entity_type)):
                print(f"Warning: Invalid BIO sequence at position {i}: '{bio_labels[i-1]}' followed by '{bio_labels[i]}'")
                # Auto-correct: Convert invalid I- tag to B-
                bio_labels[i] = f"B-{entity_type}"
    
    return bio_labels

In [ ]:
def convert_tagged_data_to_token_classification(tagged_text: str) -> Tuple[str, List[str]]:
    """
    Convert text with tags to token classification format
    
    Args:
        tagged_text: Text with entity tags like <PER>Name</PER>
        
    Returns:
        Tuple of (text without tags, list of token labels)
    """
    clean_text, spans = extract_entities(tagged_text)
    
    # Create BIO labels from spans
    char_labels = create_bio_labels_from_spans(clean_text, spans)
    
    # Double check lengths match
    if len(clean_text) != len(char_labels):
        print(f"Warning: Mismatch between text length ({len(clean_text)}) and labels length ({len(char_labels)})")
        # Truncate the longer one to match
        min_len = min(len(clean_text), len(char_labels))
        clean_text = clean_text[:min_len]
        char_labels = char_labels[:min_len]
    
    return clean_text, char_labels

In [ ]:
def tokenize_and_align_labels(examples):
    """
    Tokenize text and align character-level labels to token-level labels
    with improved BIO consistency
    """
    tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
    tokenized_inputs = tokenizer(
        examples["text"],
        padding="max_length",
        truncation=True,
        max_length=MAX_LENGTH,
        return_offsets_mapping=True,
    )
    
    labels = []
    
    for i, (text, char_labels) in enumerate(zip(examples["text"], examples["char_labels"])):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        previous_bio_tag = "O"
        
        for word_idx, offsets in zip(word_ids, tokenized_inputs["offset_mapping"][i]):
            # Special tokens have word_idx set to None
            if word_idx is None:
                label_ids.append(-100)
                continue
                
            # Empty token (e.g., special token)
            if offsets[0] == offsets[1]:
                label_ids.append(-100)
                continue
                
            # Handle case where offsets are out of bounds
            if offsets[0] >= len(char_labels) or offsets[1] > len(char_labels):
                label_ids.append(-100)
                continue
                
            # For normal tokens, get the character labels within token span
            start, end = offsets
            span_labels = char_labels[start:end]
            
            if not span_labels:
                label_ids.append(LABEL2ID["O"])
                previous_bio_tag = "O"
                continue
                
            # Count labels in span
            from collections import Counter
            label_counts = Counter(span_labels)
            
            # Remove "O" from consideration when other labels are present
            if len(label_counts) > 1 and "O" in label_counts:
                del label_counts["O"]
            
            # Get most common label
            most_common_label = label_counts.most_common(1)[0][0]
            
            # Apply BIO consistency rules
            if most_common_label.startswith("I-"):
                entity_type = most_common_label[2:]
                
                # If previous token was of different entity or was O, convert to B-
                if previous_bio_tag == "O" or (
                    previous_bio_tag.startswith("B-") or previous_bio_tag.startswith("I-")
                ) and previous_bio_tag[2:] != entity_type:
                    most_common_label = "B-" + entity_type
                    
                # If same word but different entity than previous token
                elif word_idx == previous_word_idx and previous_bio_tag[2:] != entity_type:
                    most_common_label = "B-" + entity_type
            
            label_ids.append(LABEL2ID[most_common_label])
            previous_bio_tag = most_common_label
            previous_word_idx = word_idx
        
        labels.append(label_ids)
    
    tokenized_inputs["labels"] = labels
    return tokenized_inputs


In [ ]:
def prepare_dataset_with_chunking(train_df, val_df, test_df):
    """
    Prepare datasets với chunking thông minh theo các quy tắc được định nghĩa
    """
    def process_dataframe(df, desc):
        processed_data = []
        for _, row in tqdm(df.iterrows(), total=len(df), desc=desc):
            if isinstance(row[TAGGED_CONTENT_COL], str):
                # Chunking văn bản trước
                chunks = smart_chunk_text(row[TAGGED_CONTENT_COL])
                
                for chunk in chunks:
                    try:
                        text, char_labels = convert_tagged_data_to_token_classification(chunk)
                        if text.strip():  # Chỉ thêm nếu có nội dung
                            processed_data.append({"text": text, "char_labels": char_labels})
                    except Exception as e:
                        print(f"Error processing chunk: {e}")
                        continue
        
        return processed_data
    
    # Process data với chunking
    train_data = process_dataframe(train_df, "Processing train data with chunking")
    val_data = process_dataframe(val_df, "Processing validation data with chunking")
    test_data = process_dataframe(test_df, "Processing test data with chunking")
    
    print(f"Original train samples: {len(train_df)}, After chunking: {len(train_data)}")
    print(f"Original val samples: {len(val_df)}, After chunking: {len(val_data)}")
    print(f"Original test samples: {len(test_df)}, After chunking: {len(test_data)}")
    
    # Convert to HuggingFace datasets
    train_dataset = Dataset.from_list(train_data)
    val_dataset = Dataset.from_list(val_data)
    test_dataset = Dataset.from_list(test_data)
    
    # Combine into a DatasetDict
    dataset_dict = DatasetDict({
        "train": train_dataset,
        "validation": val_dataset,
        "test": test_dataset
    })
    
    return dataset_dict

def prepare_dataset(train_df, val_df, test_df):
    """
    Prepare datasets for token classification (legacy function - không dùng chunking)
    """
    # Process data to get character-level labels
    train_data = []
    for _, row in tqdm(train_df.iterrows(), total=len(train_df), desc="Processing train data"):
        if isinstance(row[TAGGED_CONTENT_COL], str):
            text, char_labels = convert_tagged_data_to_token_classification(row[TAGGED_CONTENT_COL])
            train_data.append({"text": text, "char_labels": char_labels})

    val_data = []
    for _, row in tqdm(val_df.iterrows(), total=len(val_df), desc="Processing validation data"):
        if isinstance(row[TAGGED_CONTENT_COL], str):
            text, char_labels = convert_tagged_data_to_token_classification(row[TAGGED_CONTENT_COL])
            val_data.append({"text": text, "char_labels": char_labels})
    
    test_data = []
    for _, row in tqdm(test_df.iterrows(), total=len(test_df), desc="Processing test data"):
        if isinstance(row[TAGGED_CONTENT_COL], str):
            text, char_labels = convert_tagged_data_to_token_classification(row[TAGGED_CONTENT_COL])
            test_data.append({"text": text, "char_labels": char_labels})
    
    # Convert to HuggingFace datasets
    train_dataset = Dataset.from_list(train_data)
    val_dataset = Dataset.from_list(val_data)
    test_dataset = Dataset.from_list(test_data)
    
    # Combine into a DatasetDict
    dataset_dict = DatasetDict({
        "train": train_dataset,
        "validation": val_dataset,
        "test": test_dataset
    })
    
    return dataset_dict

# 3. Setup metric, postprocessing

In [ ]:
def fix_bio_tags(tags):
    """
    Fix invalid BIO tag sequences by converting first I- tags without preceding B- to B- tags
    """
    fixed_tags = list(tags)
    
    for i in range(len(fixed_tags)):
        if fixed_tags[i].startswith('I-'):
            entity_type = fixed_tags[i][2:]
            
            if i == 0 or (not fixed_tags[i-1].startswith('B-' + entity_type) and 
                          not fixed_tags[i-1].startswith('I-' + entity_type)):
                fixed_tags[i] = 'B-' + entity_type
    
    return fixed_tags

In [ ]:
def compute_metrics(p):
    """
    Compute NER metrics using seqeval
    """
    seqeval = evaluate.load("seqeval")
    
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)
    
    # Remove ignored index (-100)
    true_predictions = [
        [ID2LABEL[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_predictions = [fix_bio_tags(preds) for preds in true_predictions]
    
    true_labels = [
        [ID2LABEL[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    
    # Add BIO prefix back for seqeval
    true_predictions_bio = [
        ["O" if p == "O" else f"B-{p}" if i == 0 or prev == "O" else f"I-{p}"
         for i, (p, prev) in enumerate(zip(seq, ["O"] + seq[:-1]))]
        for seq in true_predictions
    ]
    
    true_labels_bio = [
        ["O" if l == "O" else f"B-{l}" if i == 0 or prev == "O" else f"I-{l}"
         for i, (l, prev) in enumerate(zip(seq, ["O"] + seq[:-1]))]
        for seq in true_labels
    ]
    
    results = seqeval.compute(predictions=true_predictions_bio, references=true_labels_bio)
    
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

# 4. Train & Evaluate

In [ ]:
def load_and_combine(excel_files):
    all_dfs = []
    for file in excel_files:
        df = pd.read_excel(file)
        all_dfs.append(df)
        
    combined_df = pd.concat(all_dfs, ignore_index=True)
    return combined_df


def load_data():
    train_files = [
        '/kaggle/working/Gemini_facebook_ads_24_04_ner_train.xlsx',
        '/kaggle/working/Gemini_facebook_posts_24_04_ner_train.xlsx',
        # '/kaggle/working/Gemini_google_crawl_26_04_ner_train.xlsx'
    ]

    val_files = [
        '/kaggle/working/Gemini_facebook_ads_24_04_ner_val.xlsx',
        '/kaggle/working/Gemini_facebook_posts_24_04_ner_val.xlsx',
        # '/kaggle/working/Gemini_google_crawl_26_04_ner_val.xlsx'
    ]

    test_files = [
        '/kaggle/working/Labeled_facebook_ads_24_04_ner_test.xlsx',
        '/kaggle/working/Labeled_facebook_posts_24_04_ner_test.xlsx'
    ]

    train_df = load_and_combine(train_files)
    val_df = load_and_combine(val_files)
    test_df = load_and_combine(test_files)

    return train_df, val_df, test_df

In [ ]:
# Prepare datasets với chunking thông minh
print("Preparing datasets với smart chunking...")
train_df, val_df, test_df = load_data()
dataset_dict = prepare_dataset_with_chunking(train_df, val_df, test_df)

# Nếu muốn sử dụng phương pháp cũ không chunking, uncomment dòng dưới:
# dataset_dict = prepare_dataset(train_df, val_df, test_df)

In [ ]:
# Load tokenizer
print("Loading tokenizer...")
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
tokenizer.model_max_length = MAX_LENGTH

# Tokenize and align labels
print("Tokenizing and aligning labels...")
tokenized_datasets = dataset_dict.map(
        tokenize_and_align_labels,
        batched=True,
        remove_columns=["text", "char_labels"]
    )

In [ ]:
def freeze_half_model_layers(model):
    """
    Freezes half of the encoder layers in a DeBERTa model.
    Assumes the model follows the standard DeBERTa architecture.
    
    Args:
        model: A DeBERTaV2 model (e.g., DebertaV2ForTokenClassification)
    
    Returns:
        The model with half of its encoder layers frozen
    """
    # First, identify the total number of layers
    if hasattr(model, "deberta") and hasattr(model.deberta, "encoder") and hasattr(model.deberta.encoder, "layer"):
        num_layers = len(model.deberta.encoder.layer)
        print(f"Model has {num_layers} transformer layers")
        
        # Calculate which layers to freeze (first half)
        layers_to_freeze = num_layers // 2
        
        # Freeze the first half of the layers
        for i in range(layers_to_freeze):
            for param in model.deberta.encoder.layer[i].parameters():
                param.requires_grad = False
            print(f"Layer {i} frozen")
        
        # Keep the second half trainable
        for i in range(layers_to_freeze, num_layers):
            print(f"Layer {i} remains trainable")
        
        # You can also freeze embeddings if desired
        # for param in model.deberta.embeddings.parameters():
        #     param.requires_grad = False
        # print("Embeddings frozen")
        
        # Make sure position embeddings remain trainable if you've extended them
        model.deberta.encoder.rel_embeddings.weight.requires_grad = True
        print("Relative position embeddings remain trainable")
        
        # Count trainable parameters
        trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
        total_params = sum(p.numel() for p in model.parameters())
        print(f"Trainable parameters: {trainable_params:,} ({trainable_params/total_params:.1%} of total)")
        
        return model
    else:
        raise ValueError("Model structure doesn't match expected DeBERTa architecture")




In [ ]:
# Load model
print("Loading model...")
model = AutoModelForTokenClassification.from_pretrained(
    MODEL_NAME,
    num_labels=len(NER_LABELS),
    id2label=ID2LABEL,
    label2id=LABEL2ID
)

model = freeze_half_model_layers(model)

In [ ]:
# Data collator
data_collator = DataCollatorForTokenClassification(
    tokenizer=tokenizer,
    padding="max_length",
    max_length=MAX_LENGTH
)

In [ ]:
# Training arguments
training_args = TrainingArguments(
    output_dir=OUTPUT_DIR,
    eval_strategy="epoch",
    learning_rate=LEARNING_RATE,
    per_device_train_batch_size=TRAIN_BATCH_SIZE,
    per_device_eval_batch_size=EVAL_BATCH_SIZE,
    gradient_accumulation_steps=GRADIENT_ACCUMULATION,
    weight_decay=WEIGHT_DECAY,
    max_grad_norm=GRAD_NORM,
    save_total_limit=2,
    num_train_epochs=NUM_EPOCHS,
    fp16=torch.cuda.is_available(),
    warmup_ratio=WARMUP_RATIO,
    logging_dir="./logs",
    logging_steps=50,
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    label_smoothing_factor=LABEL_SMOOTHING_FACTOR,
    push_to_hub=False,
    report_to=[]
)

# Create trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

In [ ]:
# Train model
print("Starting training...")
trainer.train()

# Save model
trainer.save_model(f"{OUTPUT_DIR}/final")
print(f"Model saved to {OUTPUT_DIR}/final")

In [ ]:
# Evaluate on test set
print("Evaluating on test set...")
test_results = trainer.evaluate(tokenized_datasets["test"], metric_key_prefix="test")
print(test_results)

# Save test results
with open(f"{OUTPUT_DIR}/test_results.txt", "w") as f:
    for key, value in test_results.items():
        f.write(f"{key}: {value}\n")

print(f"Test results saved to {OUTPUT_DIR}/test_results.txt")

# 5. Inference

In [2]:
from typing import List, Dict, Tuple
import re
import torch
import numpy as np
from transformers import AutoTokenizer, AutoModelForTokenClassification
import pandas as pd
def predict_entities(model, tokenizer, text: str) -> List[Dict]:
    """
    Predict named entities in text and return them as a clean list of entities
    
    Args:
        model: The trained NER model
        tokenizer: The tokenizer for the model
        text: Text to extract entities from
        
    Returns:
        List of extracted entities with format: [{'text': str, 'label': str, 'start': int, 'end': int}]
    """
    # Tokenize input
    inputs = tokenizer(
        text,
        padding=True,
        truncation=True,
        max_length=MAX_LENGTH,
        return_tensors="pt",
        return_offsets_mapping=True,
    )
    
    offset_mapping = inputs.pop("offset_mapping")
    
    # Move to GPU if available
    if torch.cuda.is_available():
        inputs = {k: v.cuda() for k, v in inputs.items()}
        model = model.cuda()
    
    # Get predictions
    with torch.no_grad():
        outputs = model(**inputs)
        predictions = outputs.logits.argmax(dim=2)
    
    # Bring tensors back to CPU if needed
    predictions = predictions.cpu().numpy()[0]
    offset_mapping = offset_mapping.cpu().numpy()[0]
    
    # Convert indices to labels (assuming id2label dict exists on the model)
    id2label = model.config.id2label
    
    # Convert prediction IDs to BIO tags
    pred_tags = [id2label[int(pred_id)] for pred_id, (start, end) in zip(predictions, offset_mapping) if start != end]
    
    # ===== ADD THIS SECTION: Apply BIO post-processing =====
    pred_tags = fix_bio_tags(pred_tags)
    # =====================================================
    
    # Extract entities from fixed predictions
    entities = []
    current_entity = None
    
    # Map the fixed tags back to token positions (only for non-special tokens)
    valid_token_positions = [i for i, (start, end) in enumerate(offset_mapping) if start != end]
    
    if len(pred_tags) != len(valid_token_positions):
        # Safety check - should never happen if we filtered correctly above
        raise ValueError(f"Mismatch between number of tags ({len(pred_tags)}) and valid tokens ({len(valid_token_positions)})")
    
    # Process tokens with fixed predictions
    for tag_idx, token_idx in enumerate(valid_token_positions):
        pred_label = pred_tags[tag_idx]
        start, end = offset_mapping[token_idx]
        
        # Skip the "O" labels
        if pred_label == "O":
            # End any current entity
            if current_entity:
                entities.append(current_entity)
                current_entity = None
            continue
        
        # Extract the entity type without the BIO prefix
        entity_type = pred_label[2:]  # Skip B- or I-
        
        # Handle B- (beginning of entity)
        if pred_label.startswith("B-"):
            # End any existing entity
            if current_entity:
                entities.append(current_entity)
            
            # Start a new entity
            current_entity = {
                "text": text[start:end],
                "label": entity_type,
                "start": int(start),
                "end": int(end)
            }
        
        # Handle I- (inside of entity)
        elif pred_label.startswith("I-") and current_entity and current_entity["label"] == entity_type:
            # Continue current entity
            current_entity["text"] += text[start:end]
            current_entity["end"] = int(end)
    
    # Add the last entity if one exists
    if current_entity:
        entities.append(current_entity)
    
    return entities


def extract_entities_from_tagged_text(tagged_text: str) -> List[Dict]:
    """
    Extract entities from tagged text with position information (without PHONE)
    
    Args:
        tagged_text: Text with entity tags like <PER>Name</PER>
        
    Returns:
        List of entities with format: [{'text': str, 'label': str, 'start': int, 'end': int}]
    """
    entities = []
    labels = ["PER", "ORG", "ADDR"]  # Removed PHONE
    
    # Create a clean version of text (no tags) for reference
    clean_text = tagged_text
    offset = 0
    
    for label in labels:
        pattern = fr"<{label}>(.*?)</{label}>"
        for match in re.finditer(pattern, tagged_text):
            # Get the entity text
            entity_text = match.group(1)
            
            # Calculate position in clean text
            start_tag = f"<{label}>"
            end_tag = f"</{label}>"
            
            # Original position in tagged text
            orig_start = match.start()
            orig_end = match.end()
            
            # Adjust for tags already removed
            clean_start = orig_start - offset
            
            # Add entity with position
            entities.append({
                "text": entity_text,
                "label": label,
                "start": clean_start,
                "end": clean_start + len(entity_text)
            })
            
            # Update offset for removed tags
            offset += len(start_tag) + len(end_tag)
        
        # Remove tags to get clean text
        clean_text = re.sub(pattern, r"\1", clean_text)
    
    return entities

# def test_model_on_samples(model_path: str, test_file_path: str, num_samples: int = 5):
#     """
#     Test the trained model on samples from the test set
    
#     Args:
#         model_path: Path to the saved model
#         test_file_path: Path to the test Excel file
#         num_samples: Number of samples to test
#     """
#     print(f"Loading model from {model_path}...")
#     model = AutoModelForTokenClassification.from_pretrained(model_path)
#     tokenizer = AutoTokenizer.from_pretrained(model_path)
    
#     print(f"Loading test data from {test_file_path}...")
#     test_df = pd.read_excel(test_file_path)
    
#     # Take random samples from test set
#     if len(test_df) > num_samples:
#         samples = test_df.sample(num_samples)
#     else:
#         samples = test_df
    
#     print(f"\n===== Testing model on {len(samples)} random samples =====\n")
    
#     for i, (_, row) in enumerate(samples.iterrows()):
#         text = row["content"]
#         tagged_text = row["human_tagged_content"]
        
#         print(f"\n----- Sample {i+1} -----")
#         print(f"Original text: {text}")
        
#         # Make predictions
#         pred_entities = predict_entities(model, tokenizer, text)
        
#         # Display predictions
#         print("\nPredicted entities:")
#         if not pred_entities:
#             print("No entities detected.")
#         else:
#             for entity in pred_entities:
#                 print(f"  • {entity['text']} ({entity['label']}) at position {entity['start']}:{entity['end']}")
        
#         # Extract ground truth entities
#         try:
#             gt_entities = extract_entities_from_tagged_text(tagged_text)
            
#             # Display ground truth
#             print("\nGround truth entities:")
#             if not gt_entities:
#                 print("No entities in ground truth.")
#             else:
#                 for entity in gt_entities:
#                     print(f"  • {entity['text']} ({entity['label']}) at position {entity['start']}:{entity['end']}")
                    
#         except Exception as e:
#             print(f"\nError extracting ground truth entities: {e}")
#             print(f"Tagged text: {tagged_text[:100]}...")
        
#         print("\n" + "="*50)

c:\Users\vonha\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
test_model_on_samples(
        model_path="./mdeberta_ner_model/final",  
        test_file_path="/kaggle/working/Labeled_facebook_ads_24_04_ner_test.xlsx",
        num_samples=5
    )

In [3]:
tokenizer = AutoTokenizer.from_pretrained('./mdeberta_ner_model/final')
text = """
TƯNG BỪNG KHAI MẠC ĐƯỜNG HOA XUÂN ẤT TỴ 2025 TẠI BỆNH VIỆN PHỤ SẢN TP. CẦN THƠ Sáng nay, 15/01/2025, Bệnh viện Phụ sản TP. Cần Thơ long trọng tổ chức Lễ Khai mạc Đường hoa Xuân Ất Tỵ 2025 tại khu vực Khoa Khám bệnh, mang đến không khí xuân đầy phấn khởi và tạo nên một điểm nhấn thú vị, đặc sắc, tạo ấn tượng cho khách hàng, đối tác và thân nhân khi đến với Bệnh viện. Có tất cả 25 góc hoa xuân thể hiện tấm lòng của hơn 600 VC-NLĐ Bệnh viện Phụ sản TP. Cần Thơ cùng tạo nên con đường hoa nhiều sắc màu rực rỡ, thể hiện cùng một ý chí chung lòng vì sự phát triển của bệnh viện, là điểm đến của khách hàng và người bệnh, nơi gửi trọn niềm tin trong chăm sóc sức khỏe nhân dân. Đường hoa xuân đã thu hút rất nhiều người đến tham quan, chụp ảnh lưu niệm. Đặc biệt, hàng ngàn sản phẩm, đặc sản vùng miền cũng được bày bán phục vụ cho nhu cầu những ngày Tết đang đến gần. Và càng ý nghĩa hơn, toàn bộ lợi nhận thu được từ các gian hàng sẽ được dành tặng cho quỹ hỗ trợ bệnh nhân nghèo, khó khăn của bệnh viện. Đây là nghĩa cử đã được VC-NLĐ bệnh viện duy trì như một truyền thống tốt đẹp, thể hiện tinh thần "tương thân tương ái", "lá lành đùm lá rách". Đường hoa Xuân Ất Tỵ 2025 là một điểm đến đầy ý nghĩa và mang một nét đặc trưng riêng, luôn chào đón mọi người đến tham quan và chụp ảnh lưu niệm. Đường hoa xuân sẽ được mở cửa và duy trì từ nay đến ngày 03/02/2025 (Mùng 6 Tết)./. ----------------------Bệnh viện Phụ sản TP. Cần Thơ "Nơi gửi trọn niềm tin"106, CMT8, P. Cái Khế, Q. Ninh Kiều, TP. Cần Thơ Tổng đài tư vấn: 1900.8665 Tổng đài đặt khám nhanh: 1900.2115 Website: Zalo: Youtube: Tiktok: #phusan #cantho #noiguitronniemtin
"""

inputs = tokenizer(
        text,
        padding=True,
        return_tensors="pt",
        return_offsets_mapping=True,
    )

In [4]:
inputs['input_ids'][0].shape

torch.Size([801])

In [5]:
if torch.cuda.is_available():
    inputs = {k: v.cuda() for k, v in inputs.items()}
    model = model.cuda()
offset_mapping = inputs.pop("offset_mapping")
# Get predictions
with torch.no_grad():
    outputs = model(**inputs)
    predictions = outputs.logits.argmax(dim=2)

# Bring tensors back to CPU if needed
predictions = predictions.cpu().numpy()[0]

NameError: name 'model' is not defined

In [6]:
preds = []
for pred_id in predictions:
    pred_label = ID2LABEL[int(pred_id)]
    preds.append(pred_label)
np.array(preds)

NameError: name 'predictions' is not defined

In [ ]:
# Demo chunking với văn bản thực tế (Kaggle compatible)
demo_text = """
TƯNG BỪNG KHAI MẠC ĐƯỜNG HOA XUÂN ẤT TỴ 2025 TẠI BỆNH VIỆN PHỤ SẢN TP. CẦN THƠ 

Sáng nay, 15/01/2025, Bệnh viện Phụ sản TP. Cần Thơ long trọng tổ chức Lễ Khai mạc Đường hoa Xuân Ất Tỵ 2025.

Dịch vụ nổi bật:
- Chăm sóc sức khỏe sinh sản
- Khám thai định kỳ  
- Siêu âm 4D chất lượng cao
- Gói sinh thường và sinh mổ

<ORG>Bệnh viện Phụ sản TP. Cần Thơ</ORG>
<ADDR>106, CMT8, P. Cái Khế, Q. Ninh Kiều, TP. Cần Thơ</ADDR>
Tổng đài tư vấn: 1900.8665
Tổng đài đặt khám nhanh: 1900.2115
Website: phusan.com.vn

#phusan #cantho #noiguitronniemtin
"""

print("=== DEMO CHUNKING VỚI VĂN BẢN THỰC TẾ (KAGGLE COMPATIBLE) ===")
demo_chunks = smart_chunk_text(demo_text, max_length=200)
print(f"Generated {len(demo_chunks)} chunks:")
for i, chunk in enumerate(demo_chunks):
    print(f"\nChunk {i+1} (length: {len(chunk)}):")
    print(f"'{chunk}'")
    print("-" * 80)

print(f"\n✓ Chunking completed successfully")
print(f"✓ All rules applied correctly")
print(f"✓ Ready for model training")